<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»


# Классификация токсичных комментариев с использованием BERT

## Описание проекта

Интернет-магазин «Викишоп» добавил функцию пользовательских правок и комментариев к описаниям товаров, что привело к увеличению активности и числу отзывов. Однако для обеспечения дружественной и безопасной среды магазин хочет автоматически выявлять токсичные комментарии и отправлять их на модерацию. В проекте будет построена модель для классификации комментариев на токсичные и нетоксичные с помощью модели BERT, ориентированной на русский язык. Цель — создать инструмент с качеством классификации F1 не ниже 0.75, что позволит обеспечить высокую точность и полноту в выявлении токсичного контента.

## Подготовка

In [25]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('datasets\\toxic_comments.csv')
display(data.head()) 


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


## Выводы

In [22]:
import os
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

# Проверка на доступность GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    data['text'], data['toxic'], test_size=0.2, random_state=42
)

# Загрузка токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

# Токенизация данных
def tokenize_data(texts, labels, tokenizer, max_length=128):
    encodings = tokenizer(
        texts.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors='pt'
    )
    dataset = torch.utils.data.TensorDataset(
        encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels.tolist())
    )
    return dataset

train_dataset = tokenize_data(X_train, y_train, tokenizer)
test_dataset = tokenize_data(X_test, y_test, tokenizer)

# Создание временных директорий для логов и выходных данных
temp_output_dir = './temp_results'
temp_log_dir = './temp_logs'
os.makedirs(temp_output_dir, exist_ok=True)
os.makedirs(temp_log_dir, exist_ok=True)

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir=temp_output_dir,  # Используйте временную директорию для выходных данных
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=temp_log_dir,  # Используйте временную директорию для логов
    logging_steps=10,
    eval_strategy="epoch"
)

# Определение функции вычисления F1
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    f1 = f1_score(labels, preds)
    return {'f1': f1}

# Тренировка модели с помощью Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

# Оценка модели на тестовой выборке
eval_results = trainer.evaluate()
print(f"F1 Score: {eval_results['eval_f1']:.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FailedPreconditionError: ./temp_logs is not a directory

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны